<a href="https://colab.research.google.com/github/robertocsa/Natural-Language-Processing-NLP-/blob/main/PLN_opera%C3%A7%C3%B5es_vetoriais_entre_representacoes_de_palavras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install spacy plotly sklearn numpy
#!python -m spacy download en_core_web_md
!python -m spacy download pt_core_news_md


  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 MB 13.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [22]:
# Importando bibliotecas necessárias
import numpy as np
import plotly.graph_objects as go
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity
import spacy

# Carregando modelo de linguagem
nlp = spacy.load("pt_core_news_md")  # Certifique-se de ter o modelo "pt_core_news_md" instalado

# Função para calcular o ângulo entre dois vetores
def calcular_angulo(vetor1, vetor2):
    similaridade_cos = cosine_similarity([vetor1], [vetor2])[0, 0]
    angulo = np.arccos(np.clip(similaridade_cos, -1.0, 1.0))  # Evita erros numéricos
    return np.degrees(angulo)

# Função para normalizar um vetor
def normalizar_vetor(vetor):
    norma = np.linalg.norm(vetor)
    return vetor / norma if norma != 0 else vetor

# Função para calcular a soma e subtração vetorial
def calcular_operacoes_vetoriais(palavra1, palavra2, palavra3):
    vetor1 = normalizar_vetor(nlp(palavra1).vector)
    vetor2 = normalizar_vetor(nlp(palavra2).vector)
    soma=0
    if (palavra3 != None):
      vetor3 = normalizar_vetor(nlp(palavra3).vector)
      soma = vetor1 - vetor2 + vetor3
    else:
      soma = vetor1 + vetor2

    soma_normalizada = normalizar_vetor(soma)

    return soma, soma_normalizada

# Função para calcular a distância Euclidiana entre dois pontos em 3D
def calcular_distancia_entre_pontos(ponto1, ponto2):
    return np.linalg.norm(ponto1 - ponto2)

# Função para plotar pares de palavras em 3D
def plotar_pares_3d(pares):
    vetores = []
    labels = []
    linhas = []  # Para armazenar informações das retas
    informacoes = []  # Lista para armazenar informações de comprimento e ângulo
    vetores_normalizados = []  # Para armazenar vetores normalizados

    for palavra1, palavra2 in pares:
        vetor1 = normalizar_vetor(nlp(palavra1).vector)
        vetor2 = normalizar_vetor(nlp(palavra2).vector)
        vetores.extend([vetor1, vetor2])
        labels.extend([palavra1, palavra2])
        linhas.append((vetor1, vetor2, palavra1, palavra2))  # Armazena os pares para cálculos posteriores
        vetores_normalizados.extend([vetor1, vetor2])

    # Adicionando o ponto resultante da soma vetorial rei - homem + mulher
    soma, soma_normalizada = calcular_operacoes_vetoriais("rei", "homem", "mulher")
    vetores.append(soma_normalizada)
    labels.append("rei - homem + mulher")
    vetores_normalizados.append(soma_normalizada)

    # Adicionando o ponto resultante da soma vetorial bicicleta - pedal + motor
    soma, soma_normalizada = calcular_operacoes_vetoriais("bicicleta", "pedal", "motor")
    vetores.append(soma_normalizada)
    labels.append("bicicleta - pedal + motor")
    vetores_normalizados.append(soma_normalizada)

    # Adicionando o ponto resultante da soma vetorial homem + amizade (supostamente, deve ficar em um local perto da palavra amigo)
    soma, soma_normalizada = calcular_operacoes_vetoriais("homem", "amizade", None)
    vetores.append(soma_normalizada)
    labels.append("homem + amizade")
    vetores_normalizados.append(soma_normalizada)

    # Reduzindo a dimensionalidade para 3D (após a normalização)
    pca = PCA(n_components=3)
    vetores_reduzidos = pca.fit_transform(vetores)

    # Criando o gráfico 3D
    fig = go.Figure()

    # Adicionando os pontos
    for i, label in enumerate(labels):
        x, y, z = vetores_reduzidos[i]
        fig.add_trace(go.Scatter3d(
            x=[x],
            y=[y],
            z=[z],
            mode='markers+text',
            marker=dict(size=6),
            text=[label],
            textposition="top center",
            name=label
        ))

    # Adicionando as retas entre os pares
    for i, (vetor1, vetor2, palavra1, palavra2) in enumerate(linhas):
        x_coords = vetores_reduzidos[2 * i:2 * i + 2, 0]
        y_coords = vetores_reduzidos[2 * i:2 * i + 2, 1]
        z_coords = vetores_reduzidos[2 * i:2 * i + 2, 2]

        # Comprimento e ângulo
        comprimento = np.linalg.norm(vetor1 - vetor2)
        angulo = calcular_angulo(vetor1, vetor2)

        # Armazenando informações
        informacoes.append({
            "par": f"{palavra1} ↔ {palavra2}",
            "comprimento": comprimento,
            "angulo": angulo
        })

        # Adicionando a reta
        fig.add_trace(go.Scatter3d(
            x=x_coords,
            y=y_coords,
            z=z_coords,
            mode='lines',
            line=dict(color='blue', width=2),
            name=f"{palavra1} ↔ {palavra2} (C: {comprimento:.2f}, A: {angulo:.2f}°)"
        ))

    # Configurando o layout
    fig.update_layout(
        title="Representação Vetorial de Palavras em 3D",
        scene=dict(
            xaxis_title="Componente Principal 1",
            yaxis_title="Componente Principal 2",
            zaxis_title="Componente Principal 3",
        ),
        legend=dict(font=dict(size=10))
    )

    fig.show()

    # Exibindo informações dos pares
    print("\nInformações dos Pares de Palavras:")
    for info in informacoes:
        print(f"Par: {info['par']} | Comprimento: {info['comprimento']:.2f} | Ângulo: {info['angulo']:.2f}°")

    # Exibindo os vetores normalizados no espaço 3D
    print("\nVetores Normalizados no Espaço 3D (após PCA):")
    for i, label in enumerate(labels):
        print(f"{label}: {vetores_reduzidos[i]}")

    # Calculando as distâncias entre os pontos finais (PCA)
    print("\nDistâncias entre os pontos finais (PCA):")
    for i in range(len(vetores_reduzidos)):
        for j in range(i + 1, len(vetores_reduzidos)):
            distancia = calcular_distancia_entre_pontos(vetores_reduzidos[i], vetores_reduzidos[j])
            print(f"Distância entre '{labels[i]}' e '{labels[j]}': {distancia:.2f}")

# Lista de pares de palavras
pares_predefinidos = [
    ("rei", "rainha"),
    ("gato", "cachorro"),
    ("carro", "bicicleta"),
    ("motocicleta","motor"),
    ("maçã", "laranja"),
    ("oceano", "mar"),
    ("homem", "mulher"),
    ("canino", "felino"),
    ("pedal","engrenagem"),
    ("amizade","inimizade"),
    ("amigo","inimigo"),
]

# Função principal
def main():
    print("Gerando gráfico para pares de palavras predefinidos...")
    plotar_pares_3d(pares_predefinidos)

# Executando
if __name__ == "__main__":
    main()


Gerando gráfico para pares de palavras predefinidos...
AQUI



Informações dos Pares de Palavras:
Par: rei ↔ rainha | Comprimento: 0.89 | Ângulo: 53.12°
Par: gato ↔ cachorro | Comprimento: 0.57 | Ângulo: 33.19°
Par: carro ↔ bicicleta | Comprimento: 0.86 | Ângulo: 51.09°
Par: motocicleta ↔ motor | Comprimento: 1.07 | Ângulo: 64.53°
Par: maçã ↔ laranja | Comprimento: 0.87 | Ângulo: 51.32°
Par: oceano ↔ mar | Comprimento: 0.68 | Ângulo: 39.69°
Par: homem ↔ mulher | Comprimento: 0.83 | Ângulo: 48.73°
Par: canino ↔ felino | Comprimento: 0.00 | Ângulo: 0.02°
Par: pedal ↔ engrenagem | Comprimento: 1.13 | Ângulo: 69.07°
Par: amizade ↔ inimizade | Comprimento: 0.97 | Ângulo: 58.02°
Par: amigo ↔ inimigo | Comprimento: 1.14 | Ângulo: 69.28°

Vetores Normalizados no Espaço 3D (após PCA):
rei: [ 0.47090719 -0.204272    0.22799381]
rainha: [ 0.41004965 -0.38828052  0.37831962]
gato: [0.11013815 0.65757222 0.1071823 ]
cachorro: [0.04172764 0.59657392 0.15651819]
carro: [-0.5193064   0.10726524  0.08243373]
bicicleta: [-0.57867152 -0.10787044  0.20241257]
motoci